# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [12]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [13]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
sql_str = """SELECT contactFirstName
                  , contactLastName
                  , phone
                  , addressLine1
                  , city
                  , state
                  , postalCode
                  , creditLimit
             FROM customers c
             WHERE (c.state = 'CA'
                AND c.creditLimit > 25000);"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]
df

,contactFirstName,contactLastName,phone,addressLine1,city,state,postalCode,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,San Rafael,CA,97562,210500
1,Julie,Murphy,6505555787,5557 North Pendale Street,San Francisco,CA,94217,64600
2,Juri,Hashimoto,6505556809,9408 Furth Circle,Burlingame,CA,94217,84600
3,Julie,Young,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,90700
4,Valarie,Thompson,7605558146,361 Furth Circle,San Diego,CA,91217,105000
5,Julie,Brown,6505551386,7734 Strong St.,San Francisco,CA,94217,105000
6,Brian,Chandler,2155554369,6047 Douglas Av.,Los Angeles,CA,91003,57700
7,Sue,Frick,4085553659,3086 Ingle Ln.,San Jose,CA,94217,77600
8,Steve,Thompson,3105553722,3675 Furth Circle,Burbank,CA,94019,55400
9,Sue,Taylor,4155554312,2793 Furth Circle,Brisbane,CA,94217,60300


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [14]:
sql_str = """SELECT state
                  , AVG(creditLimit) AS avg_creditLimit
             FROM customers c
             GROUP BY c.state;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]
df

,state,avg_creditLimit
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [15]:
sql_str = """SELECT c.customerName
                  , c.CustomerNumber
                  , os.orderNumber
                  , os.status
                  , od.quantityOrdered
             FROM customers c
             JOIN orders os USING (customerNumber)
             JOIN orderdetails od USING (orderNumber);"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [16]:
sql_str = """SELECT c.customerName
                  , c.customerNumber
                  , p.productName
                  , p.productCode
                  , SUM(od.quantityOrdered) AS total_ordered
             FROM customers c
             JOIN orders os USING (customerNumber)
             JOIN orderdetails od USING (orderNumber)
             JOIN products p USING (productCode)
             GROUP BY c.customerNumber, p.productCode
             HAVING SUM(od.quantityOrdered) > 10
             ORDER BY total_ordered DESC;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df.head()

,customerName,customerNumber,productName,productCode,total_ordered
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [17]:
sql_str = """SELECT e.firstName
                  , e.lastName
                  , e.employeeNumber
                  , e.officeCode
             FROM employees e
             WHERE e.officeCode IN(
                 SELECT officeCode
                 FROM employees
                 GROUP BY officeCode
                 HAVING COUNT(employeeNumber) < 5)
             """

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df.head()

#[2, 3, 5, 6, 7] office codes

,firstName,lastName,employeeNumber,officeCode
0,William,Patterson,1088,6
1,Julie,Firrelli,1188,2
2,Steve,Patterson,1216,2
3,Foon Yue,Tseng,1286,3
4,George,Vanauf,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!